# IMPORTS

## Libraries

In [1]:
import pandas as pd
import numpy as np
import math

pd.set_option('display.max_columns', 200)

## Load Data

In [2]:
dfRaw1 = pd.read_feather('../Data/FeatherData/YouTubeStackExchangeQueryRaw.feather')

In [3]:
dfRaw1.sample(5)

,Id,Uploader,UploadDate,Title,Description,Categories,Tags,Duration,ViewCount,LikeCount,DislikeCount,AverageRating,ThumbnailUrl,ThumbnailWidth,ThumbnailHeight
341,ck7n2B9KItQ,creativecommIT,2018-02-07,Chapter 3 Getting Started with Python and Scik...,Lecture Notes: https://rakeshgopal.teachable.c...,Education,"data science, python, scikit, scikit learn, pa...",307.0,133,2.0,NaN,5.000000,https://i.ytimg.com/vi/ck7n2B9KItQ/hqdefault.j...,168,94
2215,ljxhwHqWaYw,AiBeing Programming School,2020-05-24,"Fire Detection Using Python OpenCV, CNN - Kera...",A deep learning models to detect fire uses ope...,People & Blogs,nan,78.0,240,2.0,NaN,5.000000,https://i.ytimg.com/vi/ljxhwHqWaYw/hqdefault.j...,168,94
1382,W2hXbqnrUyY,Sigma Coding,2019-12-20,Predicting Stock Price Movement With Random Fo...,With a high-level overview of the,Education,"Sigma Coding, Random Forests, Machine Learning...",1348.0,2,41.0,2.0,4.813953,https://i.ytimg.com/vi/W2hXbqnrUyY/hqdefault.j...,168,94
673,3k7qk__5tDY,Galit Shmueli,2016-11-30,Performance 1: Data partitioning for time series,Data partitioning is a fundamental step in pre...,Education,"forecasting, performance, time series",431.0,9,136.0,7.0,4.804196,https://i.ytimg.com/vi/3k7qk__5tDY/hqdefault.j...,168,94
2711,bbkl62dvSpM,Py4Econ Official,2019-06-16,Data Science using Python хичээл 3: Support Ve...,Уг хичээлээр Machine Learning-ийн,People & Blogs,"machine learning, data science, python, suppor...",2348.0,158,7.0,NaN,5.000000,https://i.ytimg.com/vi/bbkl62dvSpM/hqdefault.j...,168,94


# DESCRIPTION OF THE DATA

In [4]:
print(f'Number of Rows: {dfRaw1.shape[0]}')
print(f'Number of Columns: {dfRaw1.shape[1]}')

Number of Rows: 4382
Number of Columns: 15


## Fillout NA

In [32]:
dfRaw1.isnull().sum()

Id                    0
Uploader              0
UploadDate            0
Title                 0
Description         243
Categories            0
Tags                  0
Duration              0
ViewCount             0
LikeCount           476
DislikeCount       2271
AverageRating       407
ThumbnailUrl          0
ThumbnailWidth        0
ThumbnailHeight       0
dtype: int64

In [39]:
# LikeCount
dfRaw1['LikeCount'] = dfRaw1['LikeCount'].apply(lambda row: 0 if math.isnan(row) else row)

# DislikeCount
dfRaw1['DislikeCount'] = dfRaw1['DislikeCount'].apply(lambda row: 0 if math.isnan(row) else row)

# DislikeCount
dfRaw1['AverageRating'] = dfRaw1['AverageRating'].apply(lambda row: -1 if math.isnan(row) else row)

# Description
dfRaw1['Description'].fillna("no description", inplace=True)

In [40]:
dfRaw1.isnull().sum()

Id                 0
Uploader           0
UploadDate         0
Title              0
Description        0
Categories         0
Tags               0
Duration           0
ViewCount          0
LikeCount          0
DislikeCount       0
AverageRating      0
ThumbnailUrl       0
ThumbnailWidth     0
ThumbnailHeight    0
dtype: int64

# Shuffle Data and Prepare to send to Google Sheets

In [41]:
dfShuffle = dfRaw1.sample(frac=1)
dfShuffle['UploadDate'] = dfShuffle['UploadDate'].astype(str)
dfShuffle = dfShuffle.values.tolist()

# Send to Google Sheets

In [43]:
from google.oauth2 import service_account
from googleapiclient.discovery import build


# Documentation: https://developers.google.com/sheets/api/quickstart/python
SERVICE_ACCOUNT_FILE = 'D:/01-DataScience/04-Projetos/00-Git/Youtube-Video-Recommendations/Credentials/keys.json'
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

credentials = None
credentials = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)


# The ID of spreadsheet.
SAMPLE_SPREADSHEET_ID = '1uCur7jOXuLnwuwfWgoBL8mvDDvchuLf-o0X-AnOxS7s'

service = build('sheets', 'v4', credentials=credentials)


# Call the Sheets API
sheet = service.spreadsheets()

# Read Values
# result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
#                             range="YoutubeScrapeSample!A1:B11").execute()

# values = result.get('values', [])

# Write Values
request = sheet.values().update(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                               range="YoutubeScrapeSample!A2", valueInputOption="USER_ENTERED", body={"values":dfShuffle}).execute()